# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [242]:
# names us

path_prefix = "/home/vsoking/Downloads/names/yob"

def df_names_us():
    df = pd.DataFrame()
    for i in range(1880, 2021, 1):
        path = path_prefix + str(i) + ".txt"
        tmp_df = pd.read_csv(path, names=["name", "gender", "births"])
        tmp_df["year"] = i
        df = df.append(tmp_df)
        df.index = pd.RangeIndex(len(df.index))
    return df[["year", "name", "gender", "births"]]

In [65]:
d["gender"].value_counts()

F    1192115
M     828748
Name: gender, dtype: int64

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [236]:
# names fr
def df_names_fr():
    df = pd.read_csv("/home/vsoking/Downloads/nat2020.csv", delimiter=';', header=0,
                     names=["gender", "name", "year", "births"])
    
    #Mêmes dtypes pour les colonnes
    df["year"] = pd.to_numeric(df["year"], errors='coerce')
    
    #Mêmes valeurs pour la colonne 'gender'
    df["gender"] = df["gender"].map({1:'M', 2:'F'})
    
    #Les lignes avec des données inutilisables doivent être supprimées
    df = df.dropna()
    
    #Mêmes dtypes pour les colonnes
    df["year"] = df["year"].astype('int64')
    
    #Seuls les prénoms de 2 caractères et plus sont conservés
    df = df.loc[df["name"].str.len() >=2]
    
    df = df.loc[~df["name"].str.contains("_Prenoms_Rares", case=False)]
    
    #La casse des prénoms doit être identique 
    df["name"] = df["name"].str.title()
    
    #Les données sont triées à l'identique    
    df.sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True], inplace=True)
    
    #L'index du DataFrame doit aller de 0 à N-1
    df.reset_index(inplace=True)
    
    #Même ordre et mêmes noms des colonnes
    return df[["year", "name", "gender", "births"]]

d = df_names_fr()
d.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630406 entries, 0 to 630405
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   year    630406 non-null  int64 
 1   name    630406 non-null  object
 2   gender  630406 non-null  object
 3   births  630406 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 19.2+ MB


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [261]:
# taux de change
def df_taux_change(devises):
    #df = pd.read_csv("/home/vsoking/Downloads/Taux_de_change_mensuel_(Archive).csv", delimiter=';', header=2)
    df = pd.read_csv("/home/vsoking/Downloads/Taux_de_change_mensuel_(Archive).csv", delimiter=';', 
                 header=1, encoding = "ISO-8859-1", index_col=0, skiprows=[2,3,4,5])
    df.index = pd.to_datetime(df.index)
    df = df.astype(float, errors='ignore')
    df = pd.DataFrame([], columns=devises)
    return df

df = pd.read_csv("/home/vsoking/Downloads/Taux_de_change_mensuel_(Archive).csv", delimiter=';', 
                 header=1, encoding = "ISO-8859-1", index_col=0, skiprows=[2,3,4,5])
df.info()




<class 'pandas.core.frame.DataFrame'>
Index: 214 entries, 01/10/2016 to 01/01/1999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   EXR.M.CYP.EUR.SP00.A  108 non-null    object
 1   EXR.M.CYP.EUR.SP00.E  108 non-null    object
 2   EXR.M.EEK.EUR.SP00.A  144 non-null    object
 3   EXR.M.EEK.EUR.SP00.E  144 non-null    object
 4   EXR.M.LTL.EUR.SP00.A  172 non-null    object
 5   EXR.M.LTL.EUR.SP00.E  172 non-null    object
 6   EXR.M.LVL.EUR.SP00.A  159 non-null    object
 7   EXR.M.LVL.EUR.SP00.E  159 non-null    object
 8   EXR.M.MTL.EUR.SP00.A  87 non-null     object
 9   EXR.M.MTL.EUR.SP00.E  87 non-null     object
 10  EXR.M.SDD.EUR.SP00.E  85 non-null     object
 11  EXR.M.SIT.EUR.SP00.A  96 non-null     object
 12  EXR.M.SIT.EUR.SP00.E  96 non-null     object
 13  EXR.M.SKK.EUR.SP00.A  99 non-null     object
 14  EXR.M.SKK.EUR.SP00.E  99 non-null     object
 15  EXR.M.SYP.EUR.SP00.E  201 non

### Tests

In [257]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [240]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [260]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... FAIL

FAIL: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-257-e90acbc5238c>", line 37, in test_df_taux_change
    self.assertTrue((df.dtypes == 'float').all())
AssertionError: False is not true

----------------------------------------------------------------------
Ran 3 tests in 5.546s

FAILED (failures=1)
